## 智能体评估 - 模拟人类

In [1]:
import os
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from typing import Literal
from langchain_core.tools import tool
from langchain.agents import create_agent
from dotenv import load_dotenv

load_dotenv(".env", override=True)

llm = ChatDeepSeek(
    model=os.environ.get("DEEPSEEK_MODEL"),
    api_base=os.environ.get("DEEPSEEK_API_BASE"),
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    temperature=0.0,
)


@tool
def get_tuikuan_zhengce():
    """查询退款政策"""
    return "机票退款时间为购票后24小时内，且航班起飞前48小时，可全额退款。"


tools = [get_tuikuan_zhengce]

prompt = "你是一名航空公司的客服人员。"
graph = create_agent(llm, tools, system_prompt=prompt)

In [2]:
# Test

from langchain_core.messages import HumanMessage


def chat_bot(query: str):
    config = {"configurable": {"thread_id": "2"}}
    res = graph.invoke({"messages": [HumanMessage(query)]}, config)
    return res


chat_bot("你好啊！")

{'messages': [HumanMessage(content='你好啊！', additional_kwargs={}, response_metadata={}, id='0f883e29-f10e-4486-80c1-7fb45497fe87'),
  AIMessage(content='您好！欢迎联系航空公司客服。请问有什么可以帮助您的吗？比如查询航班信息、预订机票、了解行李规定，或者查询退款政策等？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 295, 'total_tokens': 326, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 295}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '572d9527-68b6-4f2f-bebe-7d04380989c2', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b9635-ba7e-75b0-bc50-23d395df3038-0', usage_metadata={'input_tokens': 295, 'output_tokens': 31, 'total_tokens': 326, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})]}

### 定义模拟的真实用户

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from dotenv import load_dotenv

load_dotenv(".env", override=True)

llm = ChatDeepSeek(
    model=os.environ.get("DEEPSEEK_MODEL"),
    api_base=os.environ.get("DEEPSEEK_API_BASE"),
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    temperature=0.0,
)


# 系统提示词模板：定义客户与航空公司客户人员的交互场景
system_prompt_template = """
你是航空公司的一名客户，你正在与一位客服人员进行交互。

{instructions}

当你结束对话时，只需要回复一个单词 'FINISHED'
"""

# 创建聊天提示模板
prompt = ChatPromptTemplate.from_messages(
    [
        # 系统消息提示
        ("system", system_prompt_template),
        # 消息占位符：用户动态插入用户与AI的消息
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# 指令：定义客户的具体身份和目标
instructions = """
你的名字是Tomie。你正在试图为一次去阿拉斯加的旅行申请退款。你希望他们退还所有的钱。这次旅行发生在5年钱。
"""

# 将指令和客户姓名部分应用到提示词模板中
prompt = prompt.partial(name="Tomie", instructions=instructions)

# 模拟用户：将提示词模板与模型连接，用于生成模拟的用户交互
simulated_user = prompt | llm

### 测试真实用户反馈

In [4]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="你好，有什么可以帮助你的吗？")]
simulated_user.invoke({"messages": messages})

AIMessage(content='你好，我需要为五年前去阿拉斯加的旅行申请退款，希望能退还全部费用。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 72, 'total_tokens': 91, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 72}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '10eb9016-a078-40a1-b9f0-9098ece66480', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b963e-f202-7aa1-bec9-e482f88590f1-0', usage_metadata={'input_tokens': 72, 'output_tokens': 19, 'total_tokens': 91, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

### 使用LangGraph来进行模拟

In [12]:
from langchain_core.messages import AIMessage


def chat_bot_node(state):
    messages = state["messages"]
    # 调用客服机器人
    if len(messages) > 0:
        chat_bot_response = chat_bot(messages[-1].content)
        response_content = chat_bot_response["messages"][-1].content
        # 使用AI消息进行响应
        return {"messages": [AIMessage(content=response_content)]}


# 模拟用户节点
def _swap_roles(messages):
    new_messages = []
    for m in messages:
        # 如果消息是AIMessage，则将其转换为HumanMessage
        if isinstance(m, AIMessage):
            new_messages.append(HumanMessage(content=m.content))
        # 否则，将其转换为 AIMessage
        else:
            new_messages.append(AIMessage(content=m.content))
    return new_messages


def simulated_user_node(state):
    messages = state["messages"]
    # 交换消息的角色（AI和Human角色互换）
    new_messages = _swap_roles(messages)
    # 调用模拟用户
    simulated_user_response = simulated_user.invoke({"messages": new_messages})
    # 使用HumanMessage进行响应
    return {"messages": [HumanMessage(content=simulated_user_response.content)]}

In [13]:
# 定义条件边，结束对话的条件


def should_continue(state):
    messages = state["messages"]
    # 如果消息数里大于10，则结束
    if len(messages) > 10:
        return "end"
    if messages[-1].content == "FINISHED":
        return "end"
    # 否则，继续对话
    return "continue"

In [14]:
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)
graph_builder.add_node("user", simulated_user_node)
graph_builder.add_node("chat_bot", chat_bot_node)

graph_builder.add_edge("chat_bot", "user")
graph_builder.add_conditional_edges(
    "user",
    should_continue,
    # 如果满足结束条件，我们将停止模拟
    # 否则，虚拟用户的消息将被发送给聊天机器人
    {"end": END, "continue": "chat_bot"},
)

# 输入将首先发送给聊天机器人
graph_builder.add_edge(START, "chat_bot")
simulation = graph_builder.compile()

In [15]:
# 模拟运行

for chunk in simulation.stream({"messages": "你好！"}):
    if END not in chunk:
        print(chunk)
        print("----------")

{'chat_bot': {'messages': [AIMessage(content='您好！欢迎联系航空公司客服。请问有什么可以帮助您的吗？', additional_kwargs={}, response_metadata={}, id='11dda329-8b11-4dff-99da-b44cb71ed51f')]}}
----------
{'user': {'messages': [HumanMessage(content='你好！我叫Tomie。我想申请退款，是五年前去阿拉斯加的那次旅行。我希望你们能退还全部费用。', additional_kwargs={}, response_metadata={}, id='0607c2ea-69ae-49ed-b5ec-551366d600c6')]}}
----------
{'chat_bot': {'messages': [AIMessage(content='Tomie，根据我们航空公司的退款政策，机票退款需要在以下两个条件同时满足的情况下才能办理：\n\n1. **购票后24小时内**\n2. **航班起飞前48小时**\n\n只有同时满足这两个条件，才能申请全额退款。\n\n您提到的是五年前的旅行，这已经远远超出了我们的退款政策规定的时间范围。通常情况下，航空公司对于如此久远的机票是不提供退款的，因为相关的财务记录和系统数据可能已经归档或清理。\n\n不过，我可以为您提供一些建议：\n1. 您是否还保留着当时的机票预订确认信息或票号？\n2. 您当时是通过什么渠道购买的机票（官网、旅行社、第三方平台）？\n3. 您是否购买了旅行保险？有些保险可能包含特殊情况下的退款条款。\n\n如果您有具体的票号信息，我可以帮您查看是否有其他可能的解决方案，但根据标准政策，五年前的机票确实无法办理退款。', additional_kwargs={}, response_metadata={}, id='d7d207ee-65e6-4138-8e82-fbd20bc65580')]}}
----------
{'user': {'messages': [HumanMessage(content='我理解你们有政策，但那次旅行对我来说非常重要，而且我确实需要这笔退款。我手头可能还留着一些预订确认的邮件，但需要找一下。